# FunctionCallingAgent

In this notebook we will look into experimenting with `FunctionCallingAgent` with Simple Calculator Tools and RAG QueryEngine Tools.

`FunctionCallingAgent` uses LLM function calling capabilities to complete the task.

[LlamaIndex YT Ref](https://www.youtube.com/watch?v=6INvyrC4WrA)

In [2]:
# !pip install llama-index
# !pip install llama-index-llms-anthropic
# !pip install llama-index-llms-mistralai
# !pip install llama-index-llms-ollama

## Simple Calculator Tools

In [3]:
from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
# from llama_index.llms.anthropic import Anthropic
# from llama_index.llms.mistralai import MistralAI
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
from llama_index.core.agent import AgentRunner

### Define Tools

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

In [5]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

In [12]:
os.environ['MISTRAL_API_KEY'] = 'YOUR MISTRALAI API KEY'

## RAG QueryEngine Tools

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-04-12 21:10:15--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.02s   

2024-04-12 21:10:15 (77.5 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-04-12 21:10:15--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connec

### Load Data

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

### Create Index

We will by default use OpenAI Embeddings for building index.

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

### Define Tools

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

#### OpenAI GPT-4

In [ ]:
llm = OpenAI(model="gpt-4")
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

Added user message to memory: Compare the revenue growth of Uber and Lyft in 2021.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth in 2021?"}
=== Function Output ===
Uber's revenue grew by 57% in 2021.
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What was Lyft's revenue growth in 2021?"}
=== Function Output ===
Lyft's revenue increased by 36% in 2021 compared to the prior year.
=== LLM Response ===
In 2021, Uber's revenue grew by 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in the same year.


In [ ]:
print(str(response))

assistant: In 2021, Uber's revenue grew by 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in the same year.


#### Anthropic Sonnet

In [ ]:
llm = Anthropic(model="claude-3-sonnet-20240229")
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

Added user message to memory: Compare the revenue growth of Uber and Lyft in 2021.
=== LLM Response ===
Okay, let's use the tools to find information on Uber and Lyft's revenue growth in 2021.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth in 2021?"}
=== Function Output ===
Uber's revenue grew by 57% in 2021.
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What was Lyft's revenue growth in 2021?"}
=== Function Output ===
Lyft's revenue increased by 36% in 2021 compared to the prior year.
=== LLM Response ===
Based on the information from the tools, Uber had stronger revenue growth of 57% in 2021, compared to Lyft's 36% revenue growth in the same year. So Uber outpaced Lyft in terms of year-over-year revenue growth rate in 2021.


In [ ]:
print(response)

assistant: Based on the information from the tools, Uber had stronger revenue growth of 57% in 2021, compared to Lyft's 36% revenue growth in the same year. So Uber outpaced Lyft in terms of year-over-year revenue growth rate in 2021.


### MistralAI

In [ ]:
llm = MistralAI(model="mistral-large-latest")
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

Added user message to memory: Compare the revenue growth of Uber and Lyft in 2021.
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What was the revenue growth of Lyft in 2021?"}
=== Function Output ===
Lyft's revenue increased by 36% in 2021 compared to the prior year.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was the revenue growth of Uber in 2021?"}
=== Function Output ===
The revenue growth of Uber in 2021 was 57%.
=== LLM Response ===
In 2021, Uber's revenue growth was 57%, while Lyft's revenue growth was 36%. Therefore, Uber had a higher revenue growth rate compared to Lyft in 2021.


In [ ]:
print(response)

assistant: In 2021, Uber's revenue growth was 57%, while Lyft's revenue growth was 36%. Therefore, Uber had a higher revenue growth rate compared to Lyft in 2021.
